In [1]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()

  Activating project at `~/Documents/eth_courses/notebooks/feasibility_analysis`


In [2]:
using LinearAlgebra

# Data

In [54]:
# for vel of 61 m/s
cessna  = (;ld_ratio=11.2, mass=1338.9, vel = 61.0)
quadcopter  = (;ld_ratio=1.25, mass=160.0, vel = 45.0, η_p=0.6, n_motor=8)
battery = (; mass=2, V = 51.8, Ah = 7.2, Ah_reality_factor=0.95, C_rate=50, η_c = 0.85, price=497)
environment = (;headwind = 30)

cost_table = (;electricity_per_kWh=0.1)


const g = 9.81

9.81

## Functions

In [4]:
lift_drag_ratio(;C_L, C_D) = C_L/ C_D

thrust_req(v) = (v.mass) / v.ld_ratio # [kg]
power_req(v) =  (thrust_req(v) * v.vel) / 370.0 # [W]
power_available(v) = power_req(v) * v.η_p

actual_power_req(v) = power_req(v) / v.η_p

power_req_per_motor(vehicle ;n_motor) =  power_req(vehicle) / n_motor # [W]

power_req_per_motor (generic function with 1 method)

## Cessna

In [5]:
thrust_req(cessna) # [kg]
power_req(cessna) # [Watt]

19.708711389961394

# D'Andrea Model

In [28]:
@show thrust_req(quadcopter) # [kg]
@show power_req(quadcopter) # [kW]
@show actual_power_req(quadcopter) # [kW]

thrust_req(quadcopter) = 128.0
power_req(quadcopter) = 27.166153846153847
actual_power_req(quadcopter) = 41.79408284023668


41.79408284023668

## Battery

In [29]:
watt_hours(b) = b.Ah * b.V
Ah_reality(b) = b.Ah * b.Ah_reality_factor
max_continuous_discharge_rate(b) = b.C_rate * b.Ah
specific_energy(b) = watt_hours(b) / (b.mass * 1000) 

specific_energy (generic function with 1 method)

In [30]:
@show Ah_reality(battery)
@show max_continuous_discharge_rate(battery) # [A]
@show specific_energy(battery) # [kW h/kg]

Ah_reality(battery) = 6.84
max_continuous_discharge_rate(battery) = 360.0
specific_energy(battery) = 0.18647999999999998


0.18647999999999998

In [31]:
energy_15_min = specific_energy(battery)*4
energy_20_min = specific_energy(battery)*3


0.5594399999999999

## Worst case energy requirement

In [32]:
ν(v,env) = env.headwind / v.vel 

energy_req_per_km(v,env) = (v.mass / (370 * v.ld_ratio * v.η_p) ) # [kW.h / kg]
energy_req(v,env, t_flight) =   (v.vel*t_flight) / (1- ν(v,env)) *  energy_req_per_km(v,env)

energy_req (generic function with 1 method)

In [33]:
@show energy_req(quadcopter,environment, 0.25) 

energy_req(quadcopter, environment, 0.25) = 17.96257796257796


17.96257796257796

In [34]:
get_n_batteries(v,env, b, t_flight) = ceil(Int,energy_req(quadcopter,environment, t_flight)  / (energy_20_min*b.mass))  

get_battery_cost(v,env, b, t_flight) = get_n_batteries(v,env, b, t_flight) * b.price

get_battery_cost (generic function with 1 method)

In [35]:
@show get_n_batteries(quadcopter,environment, battery, 0.25)
@show get_battery_cost(quadcopter,environment, battery, 0.25) #[$]

get_n_batteries(quadcopter, environment, battery, 0.25) = 17
get_battery_cost(quadcopter, environment, battery, 0.25) = 8449


8449

### Average cost per kilometer

In [36]:
avg_cost_per_kilometer(v,env,b, ct) = (ct.electricity_per_kWh /  b.η_c) *  energy_req_per_km(v,env)

avg_cost_per_kilometer (generic function with 1 method)

In [37]:
@show energy_req_per_km(quadcopter,environment)
@show avg_cost_per_kilometer(quadcopter,environment,battery, cost_table) #[$]

energy_req_per_km(quadcopter, environment) = 0.5322245322245323
avg_cost_per_kilometer(quadcopter, environment, battery, cost_table) = 0.06261465084994498


0.06261465084994498

# Multicopter first principles

In [38]:
sim = (;t_flight=15/60, t_reserve=5/60)

kWh_kg(b) = watt_hours(b) / (battery.mass * 1000.0)

kWh_kg (generic function with 1 method)

## Hover

In [55]:
f_z(v) = v.mass*g
f_y(v) = v.vel * 0.1

# f_net(v) = norm([f_z(v) ,f_y(v)])
f_net(v) = f_z(v) * 0.9
power_req(v) = (f_net(v) * v.vel) / (3600.0 * v.η_p)

thrust_kg_req(v) = f_net(v) / g
thrust_kg_req_per_motor(v) = thrust_kg_req(v) / v.n_motor

thrust_kg_req_per_motor (generic function with 1 method)

In [56]:
@show f_net(quadcopter); #[N]
@show power_req(quadcopter); #[kW]


f_net(quadcopter) = 1412.64
power_req(quadcopter) = 29.43


In [57]:
@show thrust_kg_req(quadcopter); #[kg]
@show thrust_kg_req_per_motor(quadcopter); #[kg]


thrust_kg_req(quadcopter) = 144.0
thrust_kg_req_per_motor(quadcopter) = 18.0


### Worst case energy required

In [58]:
let 
airspeed_headwind_ratio = 0.66

# battery if fully discharged in given (flight time + reserve time)
battery_kWh = kWh_kg(battery) / (sim.t_flight + sim.t_reserve)

energy_req(v) = (power_req(v) * sim.t_flight) 
worst_case_energy_req(v) = energy_req(v) /(1-airspeed_headwind_ratio)

avg_cost_per_km(v,env, b, ct) =  (ct.electricity_per_kWh /  b.η_c) * (energy_req(v) / (v.vel* sim.t_flight)) # [kW.h / kg]
worst_case_cost_per_km(v,env, b, ct) =  (ct.electricity_per_kWh /  b.η_c) * (worst_case_energy_req(v) / (v.vel* sim.t_flight)) # [kW.h / kg]
    
@show worst_case_energy_req(quadcopter)

total_battery_mass = ceil(Int, worst_case_energy_req(quadcopter) / battery_kWh )
n_batteries =  total_battery_mass/battery.mass

@show total_battery_mass
@show n_batteries 
@show avg_cost_per_km(quadcopter,environment,battery, cost_table)
@show worst_case_cost_per_km(quadcopter,environment,battery, cost_table) 

end

worst_case_energy_req(quadcopter) = 21.639705882352942
total_battery_mass = 39
n_batteries = 19.5
avg_cost_per_km(quadcopter, environment, battery, cost_table) = 0.07694117647058825
worst_case_cost_per_km(quadcopter, environment, battery, cost_table) = 0.2262975778546713


0.2262975778546713

### Power at hover - momentum theory 

In [49]:
# const ρ = 1.293
# induced_vel_momentum(v, r) = sqrt((v.mass * g) / ( 2 * ρ * π * r^2 *  v.n_motor))

# power_req_momentum(v, r) = (v.n_motor * thrust_kg_req_per_motor(v)* g * induced_vel_momentum(v, r)) / (v.η_p * 1000.0)

# let
# propeller_radius = 0.406
# @show power_req_momentum(quadcopter, propeller_radius)
# end

power_req_momentum(quadcopter, propeller_radius) = 26.305854529892798


26.305854529892798

### For specific motor - TMotor U13

In [50]:
electrical_power_req(v, power_for_thrust, airspeed_headwind_ratio) = (v.n_motor * power_for_thrust * sim.t_flight) / (1-airspeed_headwind_ratio)

electrical_power_req (generic function with 1 method)

In [53]:
let

# consider energy for 20 min (5 min reserve)
battery_kWh = kWh_kg(battery) * 3
    
power_for_thrust = 3.45
airspeed_headwind_ratio = 0.66
    
total_battery_mass = ceil(Int, electrical_power_req(quadcopter, power_for_thrust,airspeed_headwind_ratio) /  battery_kWh)
n_batteries = total_battery_mass / battery.mass

@show electrical_power_req(quadcopter, power_for_thrust, airspeed_headwind_ratio); #[kW]
@show total_battery_mass
@show n_batteries
    
end

electrical_power_req(quadcopter, power_for_thrust, airspeed_headwind_ratio) = 20.294117647058826
total_battery_mass = 37
n_batteries = 18.5


18.5